#  <span style="color:darkblue"> Initial Boundary Value PDEs </span>

*Supplemental material (Julia) for Chapters 7 of "Numerical Methods and Chemical Engineering Applications" by Dorfman and Daoutidis*

<hr style="border:6px solid black"> </hr>

# Learning Objectives

- Be able to recognize Laplace & Poisson equations
- Describe the basic steps involved in the Method of Lines (MOL)
- Apply knowledge of how discretize interior nodes
- Apply knowledge of how to discretize exterior nodes for each boundary condition (Neumann, Robin, Dirichlet)
- Formulate the system of ODEs created with MOL by interlacing variables on a rectangular grid.

<hr style="border:6px solid black"> </hr>

# Simulating a graded catalyst bed

Process engineers are often engaged in tasks centered around increasing the economic productivity of process equipment while ensuring safe operation. For the commodity chemical industry, increased conversion of reactant to product remains a desired outcome. One of the most common used continuously operating reactor in this sector is that of the catalytic packed bed reactor. In this configuration, a tubular reactor is employed. This tube is packed with catalyst covered particles and the reactant flows over the catalyst bed. In many cases, a large number of tubes will be used in parallel cooled by the same jacket. We'll just consider the single tube variant here. A simplifed depiction is given below. 

<img src="reactor_diagram.png" width="600">


One important industrial chemical is that ortho-xylene which is used to produce phthalic anhydride a common plastizer. 
The desired reaction is:
<img src="react1edited.png" width="500">

However, two underdesirable side reactions are also known to occur:

<img src="react2edited.png" width="500">

<img src="react3edited.png" width="500">

The reactor is typically operated such at a hotspot occurs in the reactor, the reactor is stable (and in turn safe), and a high degree of selectivity and conversion is achieved. A number of process parameters including pressure, temperature, residence time, cooling rate, and feed composition may be manipulated to improve reactor productivity, selectivity, conversion, or overall profitability. We'll build a simulation (based on the model in this [paper](https://pubs.acs.org/doi/abs/10.1021/ie4005699).) to investigate the how using a graded catalyst may impact reactor performance. That is to say a reactor in which differing zones are loaded with catalyst of differing activity levels. This can allow the operator to raise the effective reactor temperature while preventing thermal runaway and maintaining adequate selectivity.

## A basic descriptive model

A two-dimensional steady-state pseudohomogeneous packed bed reactor equations consist of a mass balance and an energy balance:

\\[u_s\frac{\partial c_j}{\partial z} = \frac{D}{R}\frac{\partial}{\partial R}\left(R\frac{\partial c_j}{\partial R}\right) + \sum_{i}{\nu_{ij}r_i(c,T)}  \\]

\\[u_s \rho_{f} \left(\sum_{i}{C_{p,i}c_i}\right) \frac{\partial T}{\partial z} = \frac{\Lambda}{R}\frac{\partial}{\partial R}\left(R\frac{\partial T}{\partial R}\right) + \sum_{i}{(-\Delta H_{i})r_i(c,T)}  \\]

were OX is denotes ortho-xylene, PA denotes phthalic anhydride and 

\\[j \in \{c_{OX,1}, c_{PA,1}, c_{H_2 O,1}, c_{O_2,1}, c_{CO_2,1}, c_{N_2}\}\\].

For a full review of different packed bed reactor models, the reader is directed to [this reference](https://ris.utwente.nl/ws/portalfiles/portal/6073612/t0000040.pdf).

<div class="alert alert-block alert-info">
<b>Form of the above equations:</b> The above equations are coupled convection-diffusion equations. One of the most commonly encountered forms in fluid mechanics.
</div>

<div class="alert alert-block alert-warning">
<b>Limiting cases:</b> Consider briefly under what conditions we might assume that these equations reduce to the Poisson or Laplace equations below.
</div>

1. **Poisson Equation**: \\[\nabla^2 x = f(x)\\]
2. **Laplace Equation**: \\[\nabla^2 x = 0\\]

Next, we assume a sufficient cooling flow exists to hold the reactor wall temperature constant at a fixed value of $T^c$. Then we can write a symmetry and cooling boundary condition:

\\[\left.\frac{dT}{dR}\right\vert_{R=0} = 0 \qquad \qquad \left.\frac{dT}{dR}\right\vert_{R=R_t} = -Bi(T - T^c) \\]

<div class="alert alert-block alert-info">
<b>Note:</b> The wall boundary condition ($R = R_t$) is of the Robin type and the symmetry condition ($R = 0$) is of the Neumann type.
</div>

We further assume that the reactor wall is impermeable to all species (and note that similar symmetry conditions should also hold):

\\[\left.\frac{dc_j}{dR}\right\vert_{R=0} = 0 \qquad \qquad \left.\frac{dc_j}{dR}\right\vert_{R=R_t} = 0) \\]

## Further model development

- Assume the inlet stream is well-mixed.
\\[\begin{align} c_j(z=0) &= c_{j,in} \\
     T(z=0) &= T_{in} \end{align}\\]
     
<div class="alert alert-block alert-info">
<b>Note:</b> The inlet conditions are examples of Dirichlet boundary conditions.
</div>

<div class="alert alert-block alert-warning">
<b>Activity!</b> If the type of the boundary condition isn't immediate obvious take a moment and see if you can rearrange these equations into one of the below forms. 
</div>

1. **Dirichlet Boundary Condition**: \\[x = a\\] 
2. **Neumann Boundary Condition**: \\[\hat{n}\cdot\nabla x = f(x)\\]
3. **Robin Boundary Condition**: \\[\hat{n}\cdot\nabla x + \alpha x = f(x)\\]

- All reactions are treated as pseudo-first-order. Each reaction rate may be written with relation to it's partial pressure which in turn is related to the concentration assuming an ideal gas equaton of state.

\\[r_1 = (\sigma P_{O_2}\rho_s) k_1 P_{OX} = k_1 (\sigma \rho_s)/(R_g T)^2 c_{O_2} c_{OX} \\]
\\[r_2 = (\sigma P_{O_2}\rho_s) k_2 P_{PA} = k_2 (\sigma \rho_s)/(R_g T)^2 c_{O_2} c_{PA} \\]
\\[r_3 = (\sigma P_{O_2}\rho_s) k_3 P_{OX} = k_3 (\sigma \rho_s)/(R_g T)^2 c_{O_2} c_{OX} \\]

The rate constant can be calculated using the Arrhenius relationship as such

\\[k_i = k_i^r \exp{\frac{E^r_i(T - T^r)}{T R_g T^r}}, \qquad i = \{1, 2, 3\} \\]

Individual component rates of change may then be written as:

\\[\begin{align} r_{OX} &= -r_1 + r_3 \\
r_{PA} &= r_1 - r_2 \\
r_{H_2 O} &= 3r_1 + 2r_2 + 5r_3 \\
r_{O_2} &= -3r_1 - 7.5r_2 + 10.5r_3 \\
r_{CO_2} &= 8r_2 + 8r_3 \\
r_{N_2} &= 0
\end{align}\\]

Lastly, note that the heat capacity of each species may be fit to the following polynomial:

\\[ C_{p,i} = C_{0,i} + C_{1,i}T + C_{2,i}T^2 + C_{3,i}T^3 + C_{4,i}T^4 \\]

## Method of Lines Derivation

We now make use of second-order approximations for the derivatives in the radial direction

\\[\left.\frac{\partial u}{\partial R}\right\vert_{r=R_j} = \frac{u_{j+1} - u_{j-1}}{2\Delta R} \\]
\\[\left.\frac{\partial^2 u}{\partial R^2}\right\vert_{r=R_j} = \frac{u_{j+1} - 2u_{j} + u_{j-1}}{\Delta R^2} \\]

where \\[\Delta R_{j} = R_{j+1} - R_{j}\\]

### Method of Lines Derivation - Interior
Applying these rules to the mass balance for the interior nodes, we have 

\\[ \left.\frac{\partial c_j}{\partial z}\right\vert_{R=R_j} = u_s^{-1}\left(\frac{D}{R}\frac{\partial c_j}{\partial R} + \frac{\partial^2 c_j}{\partial R^2} + \sum_{i}{\nu_{ij}r_i(c,T)}\right)  \qquad j = 2, \ldots, N_{r-1}\\]

\\[ \left.\frac{d c_j}{d z}\right\vert_{R=R_j} = u_s^{-1}\left(\frac{D}{R}\frac{c_{j+1} - c_{j-1}}{2\Delta R} + D\frac{c_{j+1} - 2c_{j} + c_{j-1}}{\Delta R^2} + \sum_{i}{\nu_{ij}r_i(c,T)}\right)  \qquad j = 2, \ldots, N_{r-1}\\]

Accordlingly, the temperature balance becomes

\\[\left.\frac{\partial T}{\partial z}\right\vert_{R=R_j} = \left(u_s\sum_{i}{f_{i}c_{pj}}\right)^{-1}\left(\frac{\Lambda}{R}\frac{\partial}{\partial R}\left(R\frac{\partial T}{\partial R}\right) + \sum_{i}{(-\Delta H_{i})r_i(c,T)}\right) \qquad j = 2, \ldots, N_{r-1}   \\]

\\[ \left.\frac{d T}{d z}\right\vert_{R=R_j} = \left(u_s\sum_{i}{f_{i}c_{pj}}\right)^{-1}\left(\frac{D}{R}\frac{T_{j+1} - T_{j-1}}{2\Delta R} + D\frac{T_{j+1} - 2T_{j} + T_{j-1}}{\Delta R^2} + \sum_{i}{\nu_{ij}r_i(c,T)}\right)  \qquad j = 2, \ldots, N_{r-1}\\]

### Method of Lines Derivation - Center
The discretization at the center node yeilds:
\\[ \frac{d \mathbf{c}_1}{d z} = u_s^{-1}\left(\frac{D}{R}\frac{\mathbf{c}_{2} - \mathbf{c}_{0}}{2\Delta R} + D\frac{c_{2} - 2c_{1} + c_{0}}{\Delta R^2} + \sum_{i}{\nu_{ij}r_i(c_1,T_1)}\right)\\]

from the boundary condition we have \\[\frac{d c_1}{d R} = 0 \rightarrow \frac{c_{2} - c_{0}}{2\Delta R} = 0 \rightarrow  c_{0} = c_{2}\\]. Substituting this in yeilds:

\\[ \frac{d c_1}{d z} = u_s^{-1}\left(\frac{2D}{\Delta R^2}(c_{2} - c_{1}) + \sum_{i}{\nu_{ij}r_i(c_1,T_1)}\right)\\]

Accordingly for the energy balance we have:

\\[ \frac{d T_1}{d z} = \left(u_s\sum_{i}{f_{i}c_{pj}}\right)^{-1}\left(\frac{\Lambda}{R}\frac{T_{2} - T_{0}}{2\Delta R} + \Lambda \frac{T_{2} - 2T_{1} + T_{0}}{\Delta R^2} + \sum_{i}{(-\Delta H_{i})r_i(c_1,T_1)}\right)\\]

from the boundary condition we have \\[\frac{d T_1}{d R} = 0 \rightarrow \frac{T_{2} - T_{0}}{2\Delta R} = 0 \rightarrow  T_{0} = T_{2}\\]. Substituting this in yeilds:

\\[ \frac{d T_1}{d z} = \left(u_s\sum_{i}{f_{i}c_{pj}}\right)^{-1}\left(\frac{2\Lambda}{\Delta R^2}(T_{2} - T_{1}) + \sum_{i}{(-\Delta H_{i})r_i(c_1,T_1)}\right)\\]

<div class="alert alert-block alert-warning">
<b>Activity!</b> This derivation proceeded by solving obtaining algebraic equations from the boundary condition, solving them analytically for the fictive node value, and then substituting this expression in. We could have instead solved a system of coupled differential and algebraic equations (referred to as a differential-algebraic system of equations or DAEs). In many cases, the algebraic equations formed may not have an analytic (closed form) solution and we must resort to solving DAEs. What type of boundary condition(s) may lead algebraic equations with no analytic solution?
</div>

### Method of Lines Derivation - Wall
The discretization at the wall node yeilds:
\\[ \frac{d c_R}{d z} = u_s^{-1}\left(\frac{D}{R}\frac{c_{R+1} - c_{R-1}}{2\Delta R} + D\frac{c_{R+1} - 2c_{R} + c_{R-1}}{\Delta R^2} + \sum_{i}{\nu_{ij}r_i(c_R,T_R)}\right)\\]

from the boundary condition we have 

\\[\frac{d c_R}{d R} = 0 \rightarrow \frac{c_{R+1} - c_{R-1}}{2\Delta R} = 0 \rightarrow  c_{R+1} = c_{R-1}.\\] 

Substituting this in yeilds:

\\[ \frac{d c_R}{d z} = u_s^{-1}\left(\frac{2D}{\Delta R^2}(c_{R-1} - c_{R+1}) + \sum_{i}{\nu_{ij}r_i(c_R,T_R)}\right)\\]

Accordingly for the energy balance we have:

\\[ \frac{d T_R}{d z} = \left(u_s\sum_{i}{f_{i}c_{pj}}\right)^{-1}\left(\frac{\Lambda}{R}\frac{T_{R+1} - T_{R-1}}{2\Delta R} + \Lambda \frac{T_{R+1} - 2T_{R} + T_{R-1}}{\Delta R^2} + \sum_{i}{(-\Delta H_{i})r_i(c_R,T_R)}\right)\\]

from the boundary condition we have 
\\[\frac{d T_R}{d R} = 0 \rightarrow \frac{T_{R+1} - T_{R-1}}{2\Delta R} = 0 \rightarrow  T_{R+1} = T_{R-1}.\\]
 
Substituting this in yeilds:

\\[ \frac{d T_R}{d z} = \left(u_s\sum_{i}{f_{i}c_{pj}}\right)^{-1}\left(\frac{2\Lambda}{\Delta R^2}(T_{R-1} - T_{R}) + \sum_{i}{(-\Delta H_{i})r_i(c_R,T_R)}\right)\\]

We now need to interlace the variables letting \\[ y = \{T_1, c_{OX,1}, c_{PA,1}, c_{H_2 O,1}, c_{O_2,1}, c_{CO_2,1}, c_{N_2}, \ldots, T_{N_R}, c_{OX,N_R}, c_{PA,N_R}, c_{H_2 O,N_R}, c_{O_2,N_R}, c_{CO_2,N_R}, c_{N_2,N_R}\}\\]

<hr style="border:6px solid black"> </hr>

In [ ]:
# Input model parameters

const k_ref = [6.519E-2; 5.698E-3; 6.442E-3]                  # Reaction rate at refence temperature
const E_ref = [113.57; 129.71; 119.68]                        # Activation energy at reference temperature
const T_ref = 600                                             # Reference temperature
const Rg = 8.3144                                             # ideal gas constant

catalyst_density = 1300                                       # Density of packed catalytic material (kg/m^3)
diff_ceoff = 0.0001
heat_coeff = 7.3871                                           # heat dispersion coefficient(kJ/m h K)

T0 = 600
P0 = 1.7                                                      # total atmospheres of pressure
biot_number = 0.8                                             # Biot number, Bi (dimensionless)
reactor_length = 9                                            # length of reactor (m)
reactor_diam = 0.0254                                         # diameter of reactor (m)

feed_rate = 0.174                                             # total molar feed rate (mol/h) 
feed_mole_frac = [0.011; 0.208; 0.781; 0.0; 0.0; 0.0]         # mole fraction


In [ ]:
#=
Define rhs of implicit Euler step
=#
function residual!(out, y1, y2, a, p)
    
    # unpack parameters
    nr, nz, delZ, P0 = p
    
    # compute step size
    dr = 1/(nr - 1)
    
    # preallocate storage
    r_rxn = zeros(3)
    r_prod = zeros(6)
    k = zeros(3)
    cp = zeros(6)
        
    # pollulate the rhs of the ODEs for the interior nodes
    for i = 2:(nr-1)
        
        T = y[7*(i-1) + 1]   
        cp .= C0 + C1*T + C2*T^2 + C3*T^3 + C4*T^4
        
        k[1] = k_ref[1]*exp(E_ref[1]*(T - T_ref[1])/(T*Rg*T_ref[1]))
        k[2] = k_ref[2]*exp(E_ref[2]*(T - T_ref[2])/(T*Rg*T_ref[2]))
        k[3] = k_ref[3]*exp(E_ref[3]*(T - T_ref[3])/(T*Rg*T_ref[3]))
        
        r_rxn[1] = activity*catalyst_density*k[1]*d.conc[1]*d.conc[4]
        r_rxn[2] = activity*catalyst_density*k[2]*d.conc[2]*d.conc[4]
        r_rxn[3] = activity*catalyst_density*k[3]*d.conc[1]*d.conc[4]

        r_prod[1] = -r_rxn[1] - r_rxn[3]
        r_prod[2] = r_rxn[1] - r_rxn[2]
        r_prod[3] = 3.0*r_rxn[1] + 2.0*r_rxn[2] + 5.0*r_rxn[3]
        r_prod[4] = -3.0*r_rxn[1] - 7.5*r_rxn[2] - 10.5*r_rxn[3]
        r_prod[5] = 8.0*r_rxn[2] + 8.0*r_rxn[3]
        r_prod[6] = 0.0
        
        prefT = delZ*
        prefC = delZ*
        
        out[7*(i-1) + 1] = y2[7*(i-1) + 1] - y1[7*(i-1) + 1] + prefT*()
        out[7*(i-1) + 2] = y2[7*(i-1) + 2] - y1[7*(i-1) + 2] + prefC*()
        out[7*(i-1) + 3] = y2[7*(i-1) + 3] - y1[7*(i-1) + 3] + prefC*()
        out[7*(i-1) + 4] = y2[7*(i-1) + 4] - y1[7*(i-1) + 4] + prefC*()
        out[7*(i-1) + 5] = y2[7*(i-1) + 5] - y1[7*(i-1) + 5] + prefC*()
        out[7*(i-1) + 6] = y2[7*(i-1) + 6] - y1[7*(i-1) + 6] + prefC*()
        out[7*(i-1) + 7] = y2[7*(i-1) + 7] - y1[7*(i-1) + 7] + prefC*()
    end
   
    # pollulate the rhs of the ODEs for the center nodes
    T = y[1]
    Tp1 = y[8]
    cp .= C0 + C1*T + C2*T^2 + C3*T^3 + C4*T^4
        
    k[1] = k_ref[1]*exp(E_ref[1]*(T - T_ref[1])/(T*Rg*T_ref[1]))
    k[2] = k_ref[2]*exp(E_ref[2]*(T - T_ref[2])/(T*Rg*T_ref[2]))
    k[3] = k_ref[3]*exp(E_ref[3]*(T - T_ref[3])/(T*Rg*T_ref[3]))
        
    r_rxn[1] = activity*catalyst_density*k[1]*d.conc[1]*d.conc[4]
    r_rxn[2] = activity*catalyst_density*k[2]*d.conc[2]*d.conc[4]
    r_rxn[3] = activity*catalyst_density*k[3]*d.conc[1]*d.conc[4]

    r_prod[1] = -r_rxn[1] - r_rxn[3]
    r_prod[2] = r_rxn[1] - r_rxn[2]
    r_prod[3] = 3.0*r_rxn[1] + 2.0*r_rxn[2] + 5.0*r_rxn[3]
    r_prod[4] = -3.0*r_rxn[1] - 7.5*r_rxn[2] - 10.5*r_rxn[3]
    r_prod[5] = 8.0*r_rxn[2] + 8.0*r_rxn[3]
    r_prod[6] = 0.0
    
    prefT = delZ*
    prefC = delZ*
    
    sourceT =
    sourceC = 
    
    out[1] = y2[1] - y1[1] + prefT*((2*heat_coeff/dr^2)*(Tp1 - T) + sourceT)
    out[2] = y2[2] - y1[2] + prefC*((2*diff_ceoff/dr^2)*(Tp1 - T) + sourceT)
    out[3] = y2[3] - y1[3] + prefC*
    out[4] = y2[4] - y1[4] + prefC*
    out[5] = y2[5] - y1[5] + prefC*
    out[6] = y2[6] - y1[6] + prefC*
    out[7] = y2[7] - y1[7] + prefC*
    
    # pollulate the rhs of the ODEs for the walls nodes
    T = y[nr - 6]   
    cp .= C0 + C1*T + C2*T^2 + C3*T^3 + C4*T^4
        
    k[1] = k_ref[1]*exp(E_ref[1]*(T - T_ref[1])/(T*Rg*T_ref[1]))
    k[2] = k_ref[2]*exp(E_ref[2]*(T - T_ref[2])/(T*Rg*T_ref[2]))
    k[3] = k_ref[3]*exp(E_ref[3]*(T - T_ref[3])/(T*Rg*T_ref[3]))
        
    r_rxn[1] = activity*catalyst_density*k[1]*d.conc[1]*d.conc[4]
    r_rxn[2] = activity*catalyst_density*k[2]*d.conc[2]*d.conc[4]
    r_rxn[3] = activity*catalyst_density*k[3]*d.conc[1]*d.conc[4]

    r_prod[1] = -r_rxn[1] - r_rxn[3]
    r_prod[2] = r_rxn[1] - r_rxn[2]
    r_prod[3] = 3.0*r_rxn[1] + 2.0*r_rxn[2] + 5.0*r_rxn[3]
    r_prod[4] = -3.0*r_rxn[1] - 7.5*r_rxn[2] - 10.5*r_rxn[3]
    r_prod[5] = 8.0*r_rxn[2] + 8.0*r_rxn[3]
    r_prod[6] = 0.0
    
    prefT = delZ*
    prefC = delZ*
    
    out[nr - 6] = y2[nr - 6] - y1[nr - 6] + prefT*((2*heat_disp/dr^2)*() + )
    out[nr - 5] = y2[nr - 5] - y1[nr - 5] + prefC*
    out[nr - 4] = y2[nr - 4] - y1[nr - 4] + prefC*
    out[nr - 3] = y2[nr - 3] - y1[nr - 3] + prefC*
    out[nr - 2] = y2[nr - 2] - y1[nr - 2] + prefC*
    out[nr - 1] = y2[nr - 1] - y1[nr - 1] + prefC*
    out[nr] = y2[nr] - y1[nr] + prefC*
    
    return nothing
end

We'll use an off the shelf implementation of Newton's method through the package NLSolve.jl. 

In [1]:
import Pkg; Pkg.add("NLsolve")
using NLsolve

nr = 10 # number of discretization points in radial direction
nv = 20 # number of discretization points in axial direction

params = (nr,nv)
function activity_profile(v)
    if v < 0.25
        0.5
    elseif v < 0.5
        0.7
    end
    return 1.0
end

function nonlinear_step_solve!(out, indx, z, initial_y, params)
    a = activity_profile(z)
    residual_func! = (out, y2) -> residual!(out, initial_y, y2, a, params)
    result = nlsolve(residual_func!, initial_y, autodiff = :forward, method = :newton)
    out[indx,:] = result.zero
    return nothing
end

function integrate_ODEs!(out, initial_y, params)
    nr, P0 = p
    
    z = 0.0
    out[1,:] = initial_y                                            # store initial condition
    for indx = 2:nz
        z += delZ                                                   # advance axial dimension value
        nonlinear_step_solve!(out, indx, z, out[indx-1,:], params)  # perform a nonlinear solve for next step
    end
    return nothing
end

LoadError: ArgumentError: Package NLsolve not found in current path:
- Run `import Pkg; Pkg.add("NLsolve")` to install the NLsolve package.


In [ ]:
# define initial condition
initial_y = zeros(7*nr)
Q = feed_rate
for i = 1:nr
    initial_y[1] = T0
    initial_y[2] = feed_mole_frac[1]/Q
    initial_y[3] = feed_mole_frac[1]/Q
    initial_y[4] = feed_mole_frac[1]/Q
    initial_y[5] = feed_mole_frac[1]/Q
    initial_y[6] = feed_mole_frac[1]/Q
    initial_y[7] = feed_mole_frac[1]/Q
end

# specify parameters
params = 

# solve the system
integrate_ODEs!(out, initial_y, params)

Extract coupled values and generate 2D plot

In [ ]:
function extract_values(out, nr, nz)
    T = zeros(nr,nz)
    cOX = zeros(nr,nz)
    cPA = zeros(nr,nz)
    cH2O = zeros(nr,nz)
    cO2 = zeros(nr,nz)
    cCO2 = zeros(nr,nz)
    cN2 = zeros(nr,nz)
    for i = 1:nr
        for j = 1:nz
            T[i,j] = out[j, 7*(i-1) + 1]
            cOX[i,j] = out[j, 7*(i-1) + 2]
            cPA[i,j] = out[j, 7*(i-1) + 3]
            cH2O[i,j] = out[j, 7*(i-1) + 4]
            cO2[i,j] = out[j, 7*(i-1) + 5]
            cCO2[i,j] = out[j, 7*(i-1) + 6]
            cN2[i,j] = out[j, 7*(i-1) + 7]
        end
    end
    return T, cOX, cPA, cH2O, cO2, cCO2, cN2
end

T, cOX, cPA, cH2O, cO2, cCO2, cN2 = extract_values(out, nr, nz)

We now generate a heatmap of the temperature versus position

In [ ]:
using Plots
heatmap(nr, nz, T, c=cgrad([:blue, :yellow, :red]), xlabel="Z", ylabel="R", title="Temperature Heatmap")

<div class="alert alert-block alert-warning">
<b>Activity!</b> In the below box, compute the average temperature, concentrations, and then conversion at each spatial position. Next try, to modifying the activity_profile and re-running the example. Can you pick out can clear trends in the behavior of the system with respect to temperature/conversion with respect to activity profile?
</div>

<hr style="border:6px solid black"> </hr>

# Questions for reflection
- For a simple, what factors influence whether a system is numerically stable?
- For IVP-PDEs, the problem size can grow quite rapidly. For a 3D systems, using 20 discretization points in each dimension leads to an 8000-by-8000 system while 256 points leads to a ~16 million by ~16 million system. As illustrated by [Jaroudi, et al.](https://www.tandfonline.com/doi/full/10.1080/00207160.2019.1613526?af=R), a large number of such systems can be solved in a few hours on standard desktop. In light of this fact, how do you think the resulting linear systems formulated and solved?
- What are some other questions that you'd expect to be important when design a chemical looping combustion system? Can you provide some ideas of how numerical methods could be used to help answer these?

<hr style="border:6px solid black"> </hr>